In [1]:
import json
import os
import statistics

In [2]:
def remove_comments(raw_lines):
    code_lines = []
    sts_comment = False
    for dline in raw_lines:
        tmp_dline = dline.strip()
        if len(tmp_dline)==0:
            continue
        if tmp_dline.startswith("//"):
            continue
        if tmp_dline.startswith("/*"):
            sts_comment = True
        if not sts_comment:
            code_lines.append(tmp_dline)
        if tmp_dline.endswith("*/"):
            sts_comment = False
    return code_lines

def count_loops(code_lines):
    cnt = 0
    for dline in code_lines:
        if dline.startswith("for (") or dline.startswith("while ("):
            cnt += 1
        elif dline.startswith("for(") or dline.startswith("while("):
            cnt += 1
    return cnt

In [3]:
json_files = os.listdir("./contract-benchmarks-master/loops_asts/")
loop_asts = []
loop_lines = []
loop_counts = []
for i in range(len(json_files)):
    print("\r# loading jsons {}/{}".format(i, len(json_files)), end="")
    p = json_files[i]
    with open("./contract-benchmarks-master/loops_asts/{}".format(p), "r", encoding="iso-8859-1") as f:
        raw_lines = f.readlines()
        if len(raw_lines)<=1:
            # skip the empty json
            continue
        raw_ast = json.loads("".join(raw_lines[4:]))
    loop_asts.append(raw_ast)
    
    with open("./contract-benchmarks-master/verified_contracts/{}".format(p.replace(".json",".sol")), "r", encoding="iso-8859-1") as f:
        dlines = f.readlines()
    clines = remove_comments(dlines)
    loop_lines.append(len(clines))
    loop_counts.append(count_loops(clines))

# loading jsons 4493/4494

In [4]:
def get_value(dd, dkl):
    cur_dd = dd
    for pk in dkl:
        if isinstance(cur_dd, dict):
            if pk in cur_dd.keys():
                if cur_dd[pk] is not None:
                    cur_dd = cur_dd[pk]
                else:
                    return None
            else:
                return None
        else:
            return None
    return cur_dd

### syntactic granuality rewrite

In [42]:
#######################################
############## loop size ##############
#######################################

list_loop_node = []
list_loop_ccc = [] # corresponding LOC
def get_loop_node(cnode):
    global list_loop_node
    if cnode is None:
        return
    if isinstance(cnode, dict):
        if "nodeType" in cnode.keys():
            if cnode["nodeType"] in ["ForStatement", "WhileStatement"]:
                list_loop_node.append(cnode)
        for dkey in cnode.keys():
            get_loop_node(cnode[dkey])
    elif isinstance(cnode, list):
        for p in cnode:
            get_loop_node(p)
    else:
        return

for i in range(len(loop_asts)):
    print("\r# processing {}/{}".format(i, len(loop_asts)), end="")
    get_loop_node(loop_asts[i])
    while len(list_loop_ccc)<len(list_loop_node):
        list_loop_ccc.append(loop_lines[i])
    
    
##########################################
############## filter group ##############
##########################################
# do this once and for all the following sts
# filtered_indices = [i for i in range(len(list_loop_ccc)) if 500<=list_loop_ccc[i] and list_loop_ccc[i]<1000]
filtered_indices = [i for i in range(len(list_loop_ccc)) if 1000<=list_loop_ccc[i]]
list_loop_node = [list_loop_node[p] for p in filtered_indices]
    

list_loop_size = []
for i in range(len(list_loop_node)):
    p = list_loop_node[i]
    if "body" in p.keys() and p["body"] is not None:
        if "statements" in p["body"].keys() and p["body"]["statements"] is not None:
            list_loop_size.append(len(p["body"]["statements"]))
            
            


print()
print("# total loop nodes: {}".format(len(list_loop_node)))
print("# valid loop nodes: {}".format(len(list_loop_size)))
print("# total loop sizes: {}".format(sum(list_loop_size)))
print("# averaged loop size: {:.2f}".format(sum(list_loop_size)/len(list_loop_size)))
print("# max loop size: {}".format(max(list_loop_size)))
print("# min loop size: {}".format(min(list_loop_size)))
print("# median loop size: {:.2f}".format(statistics.median(list_loop_size)))



# processing 4466/4467
# total loop nodes: 2686
# valid loop nodes: 2180
# total loop sizes: 5215
# averaged loop size: 2.39
# max loop size: 19
# min loop size: 0
# median loop size: 2.00


In [32]:
list_loop_size.index(122)

10626

In [44]:
#######################################
############ nesting level ############
#######################################

# list_loop_node = []
# def get_loop_node(cnode):
#     global list_loop_node
#     if cnode is None:
#         return
#     if isinstance(cnode, dict):
#         if "nodeType" in cnode.keys():
#             if cnode["nodeType"] in ["ForStatement", "WhileStatement"]:
#                 list_loop_node.append(cnode)
#         for dkey in cnode.keys():
#             get_loop_node(cnode[dkey])
#     elif isinstance(cnode, list):
#         for p in cnode:
#             get_loop_node(p)
#     else:
#         return

# for i in range(len(loop_asts)):
#     print("\r# processing {}/{}".format(i, len(loop_asts)), end="")
#     get_loop_node(loop_asts[i])

list_nesting_level1 = []
tmp_list_loop_node = list_loop_node
list_loop_node = []
for p in tmp_list_loop_node:
    if "body" in p.keys() and p["body"] is not None:
        tmp0 = len(list_loop_node)
        get_loop_node(p["body"])
        tmp1 = len(list_loop_node)
        if tmp1==tmp0:
            list_nesting_level1.append(1)
    else:
        list_nesting_level1.append(1)
        
list_nesting_level2 = []
tmp_list_loop_node = list_loop_node
list_loop_node = []
for p in tmp_list_loop_node:
    if "body" in p.keys() and p["body"] is not None:
        tmp0 = len(list_loop_node)
        get_loop_node(p["body"])
        tmp1 = len(list_loop_node)
        if tmp1==tmp0:
            list_nesting_level2.append(2)
    else:
        list_nesting_level2.append(2)
        
list_nesting_level3 = []
tmp_list_loop_node = list_loop_node
list_loop_node = []
for p in tmp_list_loop_node:
    if "body" in p.keys() and p["body"] is not None:
        tmp0 = len(list_loop_node)
        get_loop_node(p["body"])
        tmp1 = len(list_loop_node)
        if tmp1==tmp0:
            list_nesting_level3.append(3)
    else:
        list_nesting_level3.append(3)
        
list_nesting_level4 = []
tmp_list_loop_node = list_loop_node
list_loop_node = []
for p in tmp_list_loop_node:
    if "body" in p.keys() and p["body"] is not None:
        tmp0 = len(list_loop_node)
        get_loop_node(p["body"])
        tmp1 = len(list_loop_node)
        if tmp1==tmp0:
            list_nesting_level4.append(4)
    else:
        list_nesting_level4.append(4)

list_nesting_levels = list_nesting_level1 + list_nesting_level2 + list_nesting_level3 + list_nesting_level4
print()
print("# valid loop nodes: {}".format(len(list_nesting_levels)))
print("# level1: {}, {:.2f}".format(len(list_nesting_level1), len(list_nesting_level1)/len(list_nesting_levels)))
print("# level2: {}, {:.2f}".format(len(list_nesting_level2), len(list_nesting_level2)/len(list_nesting_levels)))
print("# level3: {}, {:.2f}".format(len(list_nesting_level3), len(list_nesting_level3)/len(list_nesting_levels)))
print("# level4: {}, {:.2f}".format(len(list_nesting_level4), len(list_nesting_level4)/len(list_nesting_levels)))
print("# averaged nesting level: {:.2f}".format(sum(list_nesting_levels)/len(list_nesting_levels)))
print("# max nesting level: {}".format(max(list_nesting_levels)))
print("# min nesting level: {}".format(min(list_nesting_levels)))
print("# median nesting level: {:.2f}".format(statistics.median(list_nesting_levels)))


# valid loop nodes: 2735
# level1: 2491, 0.91
# level2: 196, 0.07
# level3: 48, 0.02
# level4: 0, 0.00
# averaged nesting level: 1.11
# max nesting level: 3
# min nesting level: 1
# median nesting level: 1.00


In [43]:
########################################
############ variables used ############
########################################

# list_loop_node = []
# def get_loop_node(cnode):
#     global list_loop_node
#     if cnode is None:
#         return
#     if isinstance(cnode, dict):
#         if "nodeType" in cnode.keys():
#             if cnode["nodeType"] in ["ForStatement", "WhileStatement"]:
#                 list_loop_node.append(cnode)
#         for dkey in cnode.keys():
#             get_loop_node(cnode[dkey])
#     elif isinstance(cnode, list):
#         for p in cnode:
#             get_loop_node(p)
#     else:
#         return
    
list_var_used = []
tmp_var_set = set()
def get_var_used(cnode):
    global tmp_var_set
    if cnode is None:
        return
    if isinstance(cnode, dict):
        if "nodeType" in cnode.keys():
            if cnode["nodeType"] == "Identifier":
                v = get_value(cnode, ["typeDescriptions", "typeString"])
                if v is not None:
                    if "function" not in v:
                        tmp_var_set.add(cnode["name"])
            if cnode["nodeType"] == "memberAccess":
                v = get_value(cnode, ["typeDescriptions", "typeString"])
                if v is not None:
                    if "function" not in v:
                        tmp_var_set.add(cnode["memberName"])
        for dkey in cnode.keys():
            get_var_used(cnode[dkey])
    elif isinstance(cnode, list):
        for p in cnode:
            get_var_used(p)
    else:
        return

# for i in range(len(loop_asts)):
#     print("\r# processing {}/{}".format(i, len(loop_asts)), end="")
#     get_loop_node(loop_asts[i])

for p in list_loop_node:
    tmp_var_set = set()
    get_var_used(p)
    list_var_used.append(len(tmp_var_set))


print()
print("# total loop nodes: {}".format(len(list_loop_node)))
print("# valid loop nodes: {}".format(len(list_var_used)))
print("# total var used: {}".format(sum(list_var_used)))
print("# averaged var used: {:.2f}".format(sum(list_var_used)/len(list_var_used)))
print("# max var used: {}".format(max(list_var_used)))
print("# min var used: {}".format(min(list_var_used)))
print("# median var used: {:.2f}".format(statistics.median(list_var_used)))


# total loop nodes: 2686
# valid loop nodes: 2686
# total var used: 12612
# averaged var used: 4.70
# max var used: 25
# min var used: 2
# median var used: 4.00


In [24]:
# ############################################
# ###### loops that involve collections ######
# ############################################
# # This collection is missing some types
# # Don't use this block

# list_loop_node = []
# def get_loop_node(cnode):
#     global list_loop_node
#     if cnode is None:
#         return
#     if isinstance(cnode, dict):
#         if "nodeType" in cnode.keys():
#             if cnode["nodeType"] in ["ForStatement", "WhileStatement"]:
#                 list_loop_node.append(cnode)
#         for dkey in cnode.keys():
#             get_loop_node(cnode[dkey])
#     elif isinstance(cnode, list):
#         for p in cnode:
#             get_loop_node(p)
#     else:
#         return
    
# list_var_used = []
# tmp_var_set = set()
# def get_var_used(cnode):
#     global tmp_var_set
#     if cnode is None:
#         return
#     if isinstance(cnode, dict):
#         if "nodeType" in cnode.keys():
#             if cnode["nodeType"] == "Identifier":
#                 v = get_value(cnode, ["typeDescriptions", "typeString"])
#                 if v is not None:
#                     if "array" in v or "mapping" in v:
#                         tmp_var_set.add(cnode["name"])
#             if cnode["nodeType"] == "memberAccess":
#                 v = get_value(cnode, ["typeDescriptions", "typeString"])
#                 if v is not None:
#                     if "array" in v or "mapping" in v:
#                         tmp_var_set.add(cnode["memberName"])
#         for dkey in cnode.keys():
#             get_var_used(cnode[dkey])
#     elif isinstance(cnode, list):
#         for p in cnode:
#             get_var_used(p)
#     else:
#         return

# for i in range(len(loop_asts)):
#     print("\r# processing {}/{}".format(i, len(loop_asts)), end="")
#     get_loop_node(loop_asts[i])

# for p in list_loop_node:
#     tmp_var_set = set()
#     get_var_used(p)
#     list_var_used.append(len(tmp_var_set)>0)


# print()
# print("# total loop nodes: {}".format(len(list_loop_node)))
# print("# valid loop nodes: {}".format(len(list_var_used)))
# print("# total loops with collections: {}".format(sum(list_var_used)))
# print("# %loops with collections: {:.4f}".format(sum(list_var_used)/len(list_var_used)))

In [22]:
###############################################
###### loops that involve collections II ######
###############################################

list_loop_node = []
def get_loop_node(cnode):
    global list_loop_node
    if cnode is None:
        return
    if isinstance(cnode, dict):
        if "nodeType" in cnode.keys():
            if cnode["nodeType"] in ["ForStatement", "WhileStatement"]:
                list_loop_node.append(cnode)
        for dkey in cnode.keys():
            get_loop_node(cnode[dkey])
    elif isinstance(cnode, list):
        for p in cnode:
            get_loop_node(p)
    else:
        return
    
list_var_used = []
tmp_var_set = set()
def get_var_used(cnode):
    global tmp_var_set
    if cnode is None:
        return
    if isinstance(cnode, dict):
        if "nodeType" in cnode.keys():
            if cnode["nodeType"] == "IndexAccess":
                tmp_var_set.add(1)
        for dkey in cnode.keys():
            get_var_used(cnode[dkey])
    elif isinstance(cnode, list):
        for p in cnode:
            get_var_used(p)
    else:
        return

for i in range(len(loop_asts)):
    print("\r# processing {}/{}".format(i, len(loop_asts)), end="")
    get_loop_node(loop_asts[i])

for p in list_loop_node:
    tmp_var_set = set()
    get_var_used(p)
    list_var_used.append(len(tmp_var_set)>0)


print()
print("# total loop nodes: {}".format(len(list_loop_node)))
print("# valid loop nodes: {}".format(len(list_var_used)))
print("# total loops with collections: {}".format(sum(list_var_used)))
print("# %loops with collections: {:.4f}".format(sum(list_var_used)/len(list_var_used)))

# processing 4466/4467
# total loop nodes: 14625
# valid loop nodes: 14625
# total loops with collections: 12808
# %loops with collections: 0.8758


In [25]:
#######################################
###### loops that involve writes ######
#######################################

# list_loop_node = []
# def get_loop_node(cnode):
#     global list_loop_node
#     if cnode is None:
#         return
#     if isinstance(cnode, dict):
#         if "nodeType" in cnode.keys():
#             if cnode["nodeType"] in ["ForStatement", "WhileStatement"]:
#                 list_loop_node.append(cnode)
#         for dkey in cnode.keys():
#             get_loop_node(cnode[dkey])
#     elif isinstance(cnode, list):
#         for p in cnode:
#             get_loop_node(p)
#     else:
#         return
    
list_var_used = []
tmp_var_set = set()
def get_var_used(cnode):
    global tmp_var_set
    if cnode is None:
        return
    if isinstance(cnode, dict):
        if "nodeType" in cnode.keys():
            if cnode["nodeType"] == "Assignment":
                tmp_var_set.add(1)
        for dkey in cnode.keys():
            get_var_used(cnode[dkey])
    elif isinstance(cnode, list):
        for p in cnode:
            get_var_used(p)
    else:
        return

# for i in range(len(loop_asts)):
#     print("\r# processing {}/{}".format(i, len(loop_asts)), end="")
#     get_loop_node(loop_asts[i])

for p in list_loop_node:
    tmp_var_set = set()
    get_var_used(p)
    list_var_used.append(len(tmp_var_set)>0)


print()
print("# total loop nodes: {}".format(len(list_loop_node)))
print("# valid loop nodes: {}".format(len(list_var_used)))
print("# total loops with collections: {}".format(sum(list_var_used)))
print("# %loops with collections: {:.4f}".format(sum(list_var_used)/len(list_var_used)))


# total loop nodes: 29250
# valid loop nodes: 29250
# total loops with collections: 22308
# %loops with collections: 0.7627


In [26]:
#######################################
###### loops that involve events ######
#######################################

# list_loop_node = []
# def get_loop_node(cnode):
#     global list_loop_node
#     if cnode is None:
#         return
#     if isinstance(cnode, dict):
#         if "nodeType" in cnode.keys():
#             if cnode["nodeType"] in ["ForStatement", "WhileStatement"]:
#                 list_loop_node.append(cnode)
#         for dkey in cnode.keys():
#             get_loop_node(cnode[dkey])
#     elif isinstance(cnode, list):
#         for p in cnode:
#             get_loop_node(p)
#     else:
#         return
    
list_var_used = []
tmp_var_set = set()
def get_var_used(cnode):
    global tmp_var_set
    if cnode is None:
        return
    if isinstance(cnode, dict):
        if "nodeType" in cnode.keys():
            if cnode["nodeType"] == "EmitStatement":
                tmp_var_set.add(1)
        for dkey in cnode.keys():
            get_var_used(cnode[dkey])
    elif isinstance(cnode, list):
        for p in cnode:
            get_var_used(p)
    else:
        return

# for i in range(len(loop_asts)):
#     print("\r# processing {}/{}".format(i, len(loop_asts)), end="")
#     get_loop_node(loop_asts[i])

for p in list_loop_node:
    tmp_var_set = set()
    get_var_used(p)
    list_var_used.append(len(tmp_var_set)>0)


print()
print("# total loop nodes: {}".format(len(list_loop_node)))
print("# valid loop nodes: {}".format(len(list_var_used)))
print("# total loops with collections: {}".format(sum(list_var_used)))
print("# %loops with collections: {:.4f}".format(sum(list_var_used)/len(list_var_used)))


# total loop nodes: 29250
# valid loop nodes: 29250
# total loops with collections: 1316
# %loops with collections: 0.0450


In [37]:
###############################################
###### loops that involve function calls ######
###############################################
# (including safemath calls)

# list_loop_node = []
# def get_loop_node(cnode):
#     global list_loop_node
#     if cnode is None:
#         return
#     if isinstance(cnode, dict):
#         if "nodeType" in cnode.keys():
#             if cnode["nodeType"] in ["ForStatement", "WhileStatement"]:
#                 list_loop_node.append(cnode)
#         for dkey in cnode.keys():
#             get_loop_node(cnode[dkey])
#     elif isinstance(cnode, list):
#         for p in cnode:
#             get_loop_node(p)
#     else:
#         return
    
list_var_used = []
tmp_var_set = set()
def get_var_used(cnode):
    global tmp_var_set
    if cnode is None:
        return
    if isinstance(cnode, dict):
        if "nodeType" in cnode.keys():
            if cnode["nodeType"] == "FunctionCall":
                tmp_var_set.add(1)
        for dkey in cnode.keys():
            get_var_used(cnode[dkey])
    elif isinstance(cnode, list):
        for p in cnode:
            get_var_used(p)
    else:
        return

# for i in range(len(loop_asts)):
#     print("\r# processing {}/{}".format(i, len(loop_asts)), end="")
#     get_loop_node(loop_asts[i])

for p in list_loop_node:
    tmp_var_set = set()
    get_var_used(p)
    list_var_used.append(len(tmp_var_set)>0)


print()
print("# total loop nodes: {}".format(len(list_loop_node)))
print("# valid loop nodes: {}".format(len(list_var_used)))
print("# total loops with collections: {}".format(sum(list_var_used)))
print("# %loops with collections: {:.4f}".format(sum(list_var_used)/len(list_var_used)))


# total loop nodes: 14625
# valid loop nodes: 14625
# total loops with collections: 9599
# %loops with collections: 0.6563


In [46]:
###############################################
###### loops that involve function calls ######
###############################################
# (including safemath calls)
# (kind == functionCall only, excluding typeconversion)

# list_loop_node = []
# def get_loop_node(cnode):
#     global list_loop_node
#     if cnode is None:
#         return
#     if isinstance(cnode, dict):
#         if "nodeType" in cnode.keys():
#             if cnode["nodeType"] in ["ForStatement", "WhileStatement"]:
#                 list_loop_node.append(cnode)
#         for dkey in cnode.keys():
#             get_loop_node(cnode[dkey])
#     elif isinstance(cnode, list):
#         for p in cnode:
#             get_loop_node(p)
#     else:
#         return
    
list_var_used = []
tmp_var_set = set()
def get_var_used(cnode):
    global tmp_var_set
    if cnode is None:
        return
    if isinstance(cnode, dict):
        if "nodeType" in cnode.keys():
            if cnode["nodeType"] == "FunctionCall":
                # if cnode["kind"] != "typeConversion":
                if cnode["kind"] == "functionCall":
                    tmp_var_set.add(1)
        for dkey in cnode.keys():
            get_var_used(cnode[dkey])
    elif isinstance(cnode, list):
        for p in cnode:
            get_var_used(p)
    else:
        return

# for i in range(len(loop_asts)):
#     print("\r# processing {}/{}".format(i, len(loop_asts)), end="")
#     get_loop_node(loop_asts[i])

for p in list_loop_node:
    tmp_var_set = set()
    get_var_used(p)
    # get_var_used(p["body"])
    list_var_used.append(len(tmp_var_set)>0)


print()
print("# total loop nodes: {}".format(len(list_loop_node)))
print("# valid loop nodes: {}".format(len(list_var_used)))
print("# total loops with collections: {}".format(sum(list_var_used)))
print("# %loops with collections: {:.4f}".format(sum(list_var_used)/len(list_var_used)))


# total loop nodes: 14625
# valid loop nodes: 14625
# total loops with collections: 8284
# %loops with collections: 0.5664


In [39]:
###############################################
###### loops that involve safemath calls ######
###############################################
# (this is missing functions wrapped with different names)

# list_loop_node = []
# def get_loop_node(cnode):
#     global list_loop_node
#     if cnode is None:
#         return
#     if isinstance(cnode, dict):
#         if "nodeType" in cnode.keys():
#             if cnode["nodeType"] in ["ForStatement", "WhileStatement"]:
#                 list_loop_node.append(cnode)
#         for dkey in cnode.keys():
#             get_loop_node(cnode[dkey])
#     elif isinstance(cnode, list):
#         for p in cnode:
#             get_loop_node(p)
#     else:
#         return
    
list_var_used = []
tmp_var_set = set()
def get_var_used(cnode):
    global tmp_var_set
    if cnode is None:
        return
    if isinstance(cnode, dict):
        if "nodeType" in cnode.keys():
            if cnode["nodeType"] == "FunctionCall":
                v = get_value(cnode,["expression", "memberName"])
                u = get_value(cnode,["expression", "typeDescriptions", "typeString"])
                if v is not None and u is not None:
                    if v in ["add","sub","mul","div","mod"] and " pure " in u:
                        tmp_var_set.add(1)
        for dkey in cnode.keys():
            get_var_used(cnode[dkey])
    elif isinstance(cnode, list):
        for p in cnode:
            get_var_used(p)
    else:
        return

# for i in range(len(loop_asts)):
#     print("\r# processing {}/{}".format(i, len(loop_asts)), end="")
#     get_loop_node(loop_asts[i])

for p in list_loop_node:
    tmp_var_set = set()
    get_var_used(p)
    list_var_used.append(len(tmp_var_set)>0)


print()
print("# total loop nodes: {}".format(len(list_loop_node)))
print("# valid loop nodes: {}".format(len(list_var_used)))
print("# total loops with collections: {}".format(sum(list_var_used)))
print("# %loops with collections: {:.4f}".format(sum(list_var_used)/len(list_var_used)))


# total loop nodes: 14625
# valid loop nodes: 14625
# total loops with collections: 1938
# %loops with collections: 0.1325


In [40]:
#############################################
###### loops that involve control flow ######
#############################################

# list_loop_node = []
# def get_loop_node(cnode):
#     global list_loop_node
#     if cnode is None:
#         return
#     if isinstance(cnode, dict):
#         if "nodeType" in cnode.keys():
#             if cnode["nodeType"] in ["ForStatement", "WhileStatement"]:
#                 list_loop_node.append(cnode)
#         for dkey in cnode.keys():
#             get_loop_node(cnode[dkey])
#     elif isinstance(cnode, list):
#         for p in cnode:
#             get_loop_node(p)
#     else:
#         return
    
list_var_used = []
tmp_var_set = set()
def get_var_used(cnode):
    global tmp_var_set
    if cnode is None:
        return
    if isinstance(cnode, dict):
        if "nodeType" in cnode.keys():
            if cnode["nodeType"] == "IfStatement":
                tmp_var_set.add(1)
        for dkey in cnode.keys():
            get_var_used(cnode[dkey])
    elif isinstance(cnode, list):
        for p in cnode:
            get_var_used(p)
    else:
        return

# for i in range(len(loop_asts)):
#     print("\r# processing {}/{}".format(i, len(loop_asts)), end="")
#     get_loop_node(loop_asts[i])

for p in list_loop_node:
    tmp_var_set = set()
    get_var_used(p)
    list_var_used.append(len(tmp_var_set)>0)


print()
print("# total loop nodes: {}".format(len(list_loop_node)))
print("# valid loop nodes: {}".format(len(list_var_used)))
print("# total loops with collections: {}".format(sum(list_var_used)))
print("# %loops with collections: {:.4f}".format(sum(list_var_used)/len(list_var_used)))


# total loop nodes: 14625
# valid loop nodes: 14625
# total loops with collections: 5469
# %loops with collections: 0.3739


In [60]:
########################################
###### loops that involve require ######
########################################

# list_loop_node = []
# def get_loop_node(cnode):
#     global list_loop_node
#     if cnode is None:
#         return
#     if isinstance(cnode, dict):
#         if "nodeType" in cnode.keys():
#             if cnode["nodeType"] in ["ForStatement", "WhileStatement"]:
#                 list_loop_node.append(cnode)
#         for dkey in cnode.keys():
#             get_loop_node(cnode[dkey])
#     elif isinstance(cnode, list):
#         for p in cnode:
#             get_loop_node(p)
#     else:
#         return
    
list_var_used = []
tmp_var_set = set()
def get_var_used(cnode):
    global tmp_var_set
    if cnode is None:
        return
    if isinstance(cnode, dict):
        if "nodeType" in cnode.keys():
            if cnode["nodeType"] == "FunctionCall":
                v = get_value(cnode,["expression", "name"])
                u = get_value(cnode,["expression", "typeDescriptions", "typeIdentifier"])
                if v is not None and u is not None:
                    if v == "require" and "_function_require_" in u:
                        tmp_var_set.add(1)
        for dkey in cnode.keys():
            get_var_used(cnode[dkey])
    elif isinstance(cnode, list):
        for p in cnode:
            get_var_used(p)
    else:
        return

# for i in range(len(loop_asts)):
#     print("\r# processing {}/{}".format(i, len(loop_asts)), end="")
#     get_loop_node(loop_asts[i])

for p in list_loop_node:
    tmp_var_set = set()
    get_var_used(p)
    list_var_used.append(len(tmp_var_set)>0)


print()
print("# total loop nodes: {}".format(len(list_loop_node)))
print("# valid loop nodes: {}".format(len(list_var_used)))
print("# total loops with collections: {}".format(sum(list_var_used)))
print("# %loops with collections: {:.4f}".format(sum(list_var_used)/len(list_var_used)))


# total loop nodes: 14625
# valid loop nodes: 14625
# total loops with collections: 1899
# %loops with collections: 0.1298


In [59]:
################################################
###### loops that involve transfer (only) ######
################################################

# list_loop_node = []
# def get_loop_node(cnode):
#     global list_loop_node
#     if cnode is None:
#         return
#     if isinstance(cnode, dict):
#         if "nodeType" in cnode.keys():
#             if cnode["nodeType"] in ["ForStatement", "WhileStatement"]:
#                 list_loop_node.append(cnode)
#         for dkey in cnode.keys():
#             get_loop_node(cnode[dkey])
#     elif isinstance(cnode, list):
#         for p in cnode:
#             get_loop_node(p)
#     else:
#         return
    
list_var_used = []
tmp_var_set = set()
def get_var_used(cnode):
    global tmp_var_set
    if cnode is None:
        return
    if isinstance(cnode, dict):
        if "nodeType" in cnode.keys():
            if cnode["nodeType"] == "FunctionCall":
                v = get_value(cnode,["expression", "memberName"])
                u = get_value(cnode,["expression", "name"])
                if v is not None:
                    if "transfer" in v:
                        tmp_var_set.add(1)
                elif u is not None:
                    if "transfer" in u:
                        tmp_var_set.add(1)
        for dkey in cnode.keys():
            get_var_used(cnode[dkey])
    elif isinstance(cnode, list):
        for p in cnode:
            get_var_used(p)
    else:
        return

# for i in range(len(loop_asts)):
#     print("\r# processing {}/{}".format(i, len(loop_asts)), end="")
#     get_loop_node(loop_asts[i])

for p in list_loop_node:
    tmp_var_set = set()
    get_var_used(p)
    list_var_used.append(len(tmp_var_set)>0)


print()
print("# total loop nodes: {}".format(len(list_loop_node)))
print("# valid loop nodes: {}".format(len(list_var_used)))
print("# total loops with collections: {}".format(sum(list_var_used)))
print("# %loops with collections: {:.4f}".format(sum(list_var_used)/len(list_var_used)))


# total loop nodes: 14625
# valid loop nodes: 14625
# total loops with collections: 1100
# %loops with collections: 0.0752


In [58]:
#########################################
###### loops that involve transfer ######
#########################################
# (including all different transfers)

# list_loop_node = []
# def get_loop_node(cnode):
#     global list_loop_node
#     if cnode is None:
#         return
#     if isinstance(cnode, dict):
#         if "nodeType" in cnode.keys():
#             if cnode["nodeType"] in ["ForStatement", "WhileStatement"]:
#                 list_loop_node.append(cnode)
#         for dkey in cnode.keys():
#             get_loop_node(cnode[dkey])
#     elif isinstance(cnode, list):
#         for p in cnode:
#             get_loop_node(p)
#     else:
#         return
    
list_var_used = []
tmp_var_set = set()
def get_var_used(cnode):
    global tmp_var_set
    if cnode is None:
        return
    if isinstance(cnode, dict):
        if "nodeType" in cnode.keys():
            if cnode["nodeType"] == "FunctionCall":
                v = get_value(cnode,["expression", "memberName"])
                u = get_value(cnode,["expression", "name"])
                if v is not None:
                    if "transfer" in v or "send" in v or v == "value":
                        tmp_var_set.add(1)
                elif u is not None:
                    if "transfer" in u or "send" in u or u == "value":
                        tmp_var_set.add(1)
        for dkey in cnode.keys():
            get_var_used(cnode[dkey])
    elif isinstance(cnode, list):
        for p in cnode:
            get_var_used(p)
    else:
        return

# for i in range(len(loop_asts)):
#     print("\r# processing {}/{}".format(i, len(loop_asts)), end="")
#     get_loop_node(loop_asts[i])

for p in list_loop_node:
    tmp_var_set = set()
    get_var_used(p)
    list_var_used.append(len(tmp_var_set)>0)


print()
print("# total loop nodes: {}".format(len(list_loop_node)))
print("# valid loop nodes: {}".format(len(list_var_used)))
print("# total loops with collections: {}".format(sum(list_var_used)))
print("# %loops with collections: {:.4f}".format(sum(list_var_used)/len(list_var_used)))


# total loop nodes: 14625
# valid loop nodes: 14625
# total loops with collections: 1237
# %loops with collections: 0.0846


In [54]:
##############################################
###### loops that involve internal call ######
##############################################


# list_loop_node = []
# def get_loop_node(cnode):
#     global list_loop_node
#     if cnode is None:
#         return
#     if isinstance(cnode, dict):
#         if "nodeType" in cnode.keys():
#             if cnode["nodeType"] in ["ForStatement", "WhileStatement"]:
#                 list_loop_node.append(cnode)
#         for dkey in cnode.keys():
#             get_loop_node(cnode[dkey])
#     elif isinstance(cnode, list):
#         for p in cnode:
#             get_loop_node(p)
#     else:
#         return
    
list_var_used = []
tmp_var_set = set()
def get_var_used(cnode):
    global tmp_var_set
    if cnode is None:
        return
    if isinstance(cnode, dict):
        if "nodeType" in cnode.keys():
            if cnode["nodeType"] == "FunctionCall":
                v = get_value(cnode,["expression", "typeDescriptions", "typeIdentifier"])
                # no need to get a name because some statements are like
                # cycle.offerAccepted.value(_etherAmount)(_investor, _tokenAmount)
                # which do not necessarily have a name
                if v is not None:
                    if "_function_internal_" in v:
                        tmp_var_set.add(1)
        for dkey in cnode.keys():
            get_var_used(cnode[dkey])
    elif isinstance(cnode, list):
        for p in cnode:
            get_var_used(p)
    else:
        return

# for i in range(len(loop_asts)):
#     print("\r# processing {}/{}".format(i, len(loop_asts)), end="")
#     get_loop_node(loop_asts[i])

for p in list_loop_node:
    tmp_var_set = set()
    get_var_used(p)
    list_var_used.append(len(tmp_var_set)>0)


print()
print("# total loop nodes: {}".format(len(list_loop_node)))
print("# valid loop nodes: {}".format(len(list_var_used)))
print("# total loops with collections: {}".format(sum(list_var_used)))
print("# %loops with collections: {:.4f}".format(sum(list_var_used)/len(list_var_used)))


# total loop nodes: 14625
# valid loop nodes: 14625
# total loops with collections: 5936
# %loops with collections: 0.4059


In [69]:
#############################################
###### write: scalar, read: collection ######
#############################################

# list_loop_node = []
# def get_loop_node(cnode):
#     global list_loop_node
#     if cnode is None:
#         return
#     if isinstance(cnode, dict):
#         if "nodeType" in cnode.keys():
#             if cnode["nodeType"] in ["ForStatement", "WhileStatement"]:
#                 list_loop_node.append(cnode)
#         for dkey in cnode.keys():
#             get_loop_node(cnode[dkey])
#     elif isinstance(cnode, list):
#         for p in cnode:
#             get_loop_node(p)
#     else:
#         return

list_assignment_node = []
tmpl_assignment_node = []
def get_assignment_node(cnode):
    global tmpl_assignment_node
    if cnode is None:
        return
    if isinstance(cnode, dict):
        if "nodeType" in cnode.keys():
            if cnode["nodeType"] in ["Assignment"]:
                tmpl_assignment_node.append(cnode)
        for dkey in cnode.keys():
            get_assignment_node(cnode[dkey])
    elif isinstance(cnode, list):
        for p in cnode:
            get_assignment_node(p)
    else:
        return
    
list_var_used = []
tmp_var_set = set()
def detect_var(cnode):
    if cnode is None:
        return False
    if isinstance(cnode, dict):
        v = get_value(cnode, ["nodeType"])
        if v is not None and v == "IndexAccess":
            return True
        else:
            for dkey in cnode.keys():
                if detect_var(cnode[dkey]):
                    return True
            return False
    elif isinstance(cnode, list):
        for p in cnode:
            if detect_var(p):
                return True
        return False
    else:
        return False
    
def get_var_used(cnode):
    global tmp_var_set
    if cnode is None:
        return
    if isinstance(cnode, dict):
        # (notice) ast children are executed first, so the outer most being NOT IndexAccess is OK enough
        # e.g., object[i].acc, outermost is not IndexAccess
        v = get_value(cnode, ["leftHandSide", "nodeType"])
        if v is not None and v != "IndexAccess":
            # lhs is not collection
            u = get_value(cnode, ["rightHandSide"])
            r = detect_var(u)
            if r:
                tmp_var_set.add(1)
    
# for i in range(len(loop_asts)):
#     print("\r# processing {}/{}".format(i, len(loop_asts)), end="")
#     get_loop_node(loop_asts[i])

for i in range(len(list_loop_node)):
    print("\r# processing {}/{}".format(i, len(list_loop_node)), end="")
    tmpl_assignment_node = []
    get_assignment_node(list_loop_node[i])
    list_assignment_node.append(tmpl_assignment_node)
    

for p in list_assignment_node:
    tmp_var_set = set()
    for j in p:
        get_var_used(j)
    list_var_used.append(len(tmp_var_set)>0)


print()
print("# total loop nodes: {}".format(len(list_loop_node)))
print("# valid loop nodes: {}".format(len(list_var_used)))
print("# total loops with scalar<-collection: {}".format(sum(list_var_used)))
print("# %loops with scalar<-collection: {:.4f}".format(sum(list_var_used)/len(list_var_used)))

# processing 14624/14625
# total loop nodes: 14625
# valid loop nodes: 14625
# total loops with scalar<-collection: 2653
# %loops with scalar<-collection: 0.1814


In [71]:
#################################################
###### write: collection, read: collection ######
#################################################

# list_loop_node = []
# def get_loop_node(cnode):
#     global list_loop_node
#     if cnode is None:
#         return
#     if isinstance(cnode, dict):
#         if "nodeType" in cnode.keys():
#             if cnode["nodeType"] in ["ForStatement", "WhileStatement"]:
#                 list_loop_node.append(cnode)
#         for dkey in cnode.keys():
#             get_loop_node(cnode[dkey])
#     elif isinstance(cnode, list):
#         for p in cnode:
#             get_loop_node(p)
#     else:
#         return

list_assignment_node = []
tmpl_assignment_node = []
def get_assignment_node(cnode):
    global tmpl_assignment_node
    if cnode is None:
        return
    if isinstance(cnode, dict):
        if "nodeType" in cnode.keys():
            if cnode["nodeType"] in ["Assignment"]:
                tmpl_assignment_node.append(cnode)
        for dkey in cnode.keys():
            get_assignment_node(cnode[dkey])
    elif isinstance(cnode, list):
        for p in cnode:
            get_assignment_node(p)
    else:
        return
    
list_var_used = []
tmp_var_set = set()
def detect_var(cnode):
    if cnode is None:
        return False
    if isinstance(cnode, dict):
        v = get_value(cnode, ["nodeType"])
        if v is not None and v == "IndexAccess":
            return True
        else:
            for dkey in cnode.keys():
                if detect_var(cnode[dkey]):
                    return True
            return False
    elif isinstance(cnode, list):
        for p in cnode:
            if detect_var(p):
                return True
        return False
    else:
        return False
    
def get_var_used(cnode):
    global tmp_var_set
    if cnode is None:
        return
    if isinstance(cnode, dict):
        # (notice) ast children are executed first, so the outer most being NOT IndexAccess is OK enough
        # e.g., object[i].acc, outermost is not IndexAccess
        v = get_value(cnode, ["leftHandSide", "nodeType"])
        if v is not None and v == "IndexAccess":
            # lhs is collection
            u = get_value(cnode, ["rightHandSide"])
            r = detect_var(u)
            if r:
                tmp_var_set.add(1)
    
# for i in range(len(loop_asts)):
#     print("\r# processing {}/{}".format(i, len(loop_asts)), end="")
#     get_loop_node(loop_asts[i])

for i in range(len(list_loop_node)):
    print("\r# processing {}/{}".format(i, len(list_loop_node)), end="")
    tmpl_assignment_node = []
    get_assignment_node(list_loop_node[i])
    list_assignment_node.append(tmpl_assignment_node)
    

for p in list_assignment_node:
    tmp_var_set = set()
    for j in p:
        get_var_used(j)
    list_var_used.append(len(tmp_var_set)>0)


print()
print("# total loop nodes: {}".format(len(list_loop_node)))
print("# valid loop nodes: {}".format(len(list_var_used)))
print("# total loops with collection<-collection: {}".format(sum(list_var_used)))
print("# %loops with collection<-collection: {:.4f}".format(sum(list_var_used)/len(list_var_used)))

# processing 14624/14625
# total loop nodes: 14625
# valid loop nodes: 14625
# total loops with collection<-collection: 3477
# %loops with collection<-collection: 0.2377


In [72]:
#############################################
###### write: collection, read: scalar ######
#############################################
# specifically, the scalar should only appear on first level of RHS

# list_loop_node = []
# def get_loop_node(cnode):
#     global list_loop_node
#     if cnode is None:
#         return
#     if isinstance(cnode, dict):
#         if "nodeType" in cnode.keys():
#             if cnode["nodeType"] in ["ForStatement", "WhileStatement"]:
#                 list_loop_node.append(cnode)
#         for dkey in cnode.keys():
#             get_loop_node(cnode[dkey])
#     elif isinstance(cnode, list):
#         for p in cnode:
#             get_loop_node(p)
#     else:
#         return

list_assignment_node = []
tmpl_assignment_node = []
def get_assignment_node(cnode):
    global tmpl_assignment_node
    if cnode is None:
        return
    if isinstance(cnode, dict):
        if "nodeType" in cnode.keys():
            if cnode["nodeType"] in ["Assignment"]:
                tmpl_assignment_node.append(cnode)
        for dkey in cnode.keys():
            get_assignment_node(cnode[dkey])
    elif isinstance(cnode, list):
        for p in cnode:
            get_assignment_node(p)
    else:
        return
    
list_var_used = []
tmp_var_set = set()
def detect_var(cnode):
    # (special) if there's an IndexAccess, return False immediately
    if cnode is None:
        return False
    if isinstance(cnode, dict):
        v = get_value(cnode, ["nodeType"])
        if v is not None and v == "IndexAccess":
            return False
        if v is not None and v == "Identifier":
            t = get_value(cnode, ["typeDescriptions", "typeString"])
            if t is not None and "[]" not in t:
                return True
        else:
            for dkey in cnode.keys():
                if detect_var(cnode[dkey]):
                    return True
            return False
    elif isinstance(cnode, list):
        for p in cnode:
            if detect_var(p):
                return True
        return False
    else:
        return False
    
def get_var_used(cnode):
    global tmp_var_set
    if cnode is None:
        return
    if isinstance(cnode, dict):
        # (notice) ast children are executed first, so the outer most being NOT IndexAccess is OK enough
        # e.g., object[i].acc, outermost is not IndexAccess
        v = get_value(cnode, ["leftHandSide", "nodeType"])
        if v is not None and v == "IndexAccess":
            # lhs is collection
            u = get_value(cnode, ["rightHandSide"])
            r = detect_var(u)
            if r:
                tmp_var_set.add(1)
    
# for i in range(len(loop_asts)):
#     print("\r# processing {}/{}".format(i, len(loop_asts)), end="")
#     get_loop_node(loop_asts[i])

for i in range(len(list_loop_node)):
    print("\r# processing {}/{}".format(i, len(list_loop_node)), end="")
    tmpl_assignment_node = []
    get_assignment_node(list_loop_node[i])
    list_assignment_node.append(tmpl_assignment_node)
    

for p in list_assignment_node:
    tmp_var_set = set()
    for j in p:
        get_var_used(j)
    list_var_used.append(len(tmp_var_set)>0)


print()
print("# total loop nodes: {}".format(len(list_loop_node)))
print("# valid loop nodes: {}".format(len(list_var_used)))
print("# total loops with collection<-scalar: {}".format(sum(list_var_used)))
print("# %loops with collection<-scalar: {:.4f}".format(sum(list_var_used)/len(list_var_used)))

# processing 14624/14625
# total loop nodes: 14625
# valid loop nodes: 14625
# total loops with collection<-scalar: 2325
# %loops with collection<-scalar: 0.1590


In [73]:
#######################################
###### collection[collection[?]] ######
#######################################

# list_loop_node = []
# def get_loop_node(cnode):
#     global list_loop_node
#     if cnode is None:
#         return
#     if isinstance(cnode, dict):
#         if "nodeType" in cnode.keys():
#             if cnode["nodeType"] in ["ForStatement", "WhileStatement"]:
#                 list_loop_node.append(cnode)
#         for dkey in cnode.keys():
#             get_loop_node(cnode[dkey])
#     elif isinstance(cnode, list):
#         for p in cnode:
#             get_loop_node(p)
#     else:
#         return

list_assignment_node = []
tmpl_assignment_node = []
def get_assignment_node(cnode):
    global tmpl_assignment_node
    if cnode is None:
        return
    if isinstance(cnode, dict):
        if "nodeType" in cnode.keys():
            if cnode["nodeType"] in ["Assignment"]:
                tmpl_assignment_node.append(cnode)
        for dkey in cnode.keys():
            get_assignment_node(cnode[dkey])
    elif isinstance(cnode, list):
        for p in cnode:
            get_assignment_node(p)
    else:
        return
    
list_ia_node = []
tmpl_ia_node = []
def get_ia_node(cnode):
    global tmpl_ia_node
    if cnode is None:
        return
    if isinstance(cnode, dict):
        if "nodeType" in cnode.keys():
            if cnode["nodeType"] in ["IndexAccess"]:
                tmpl_ia_node.append(cnode)
        for dkey in cnode.keys():
            get_ia_node(cnode[dkey])
    elif isinstance(cnode, list):
        for p in cnode:
            get_ia_node(p)
    else:
        return
    
# for i in range(len(loop_asts)):
#     print("\r# processing {}/{}".format(i, len(loop_asts)), end="")
#     get_loop_node(loop_asts[i])

for i in range(len(list_loop_node)):
    print("\r# processing {}/{}".format(i, len(list_loop_node)), end="")
    tmpl_assignment_node = []
    get_assignment_node(list_loop_node[i])
    list_assignment_node.append(tmpl_assignment_node)
    

for p in list_assignment_node:
    tmpl_ia_node = []
    for j in p:
        get_ia_node(j)
    list_ia_node.append(tmpl_ia_node)
    
list_ia_node0 = list_ia_node
list_ia_node = []

for p in list_ia_node0:
    tmpl_ia_node = []
    for j in p:
        get_ia_node(j["indexExpression"])
    # list_ia_node.append(tmpl_ia_node)
    list_ia_node.append(len(tmpl_ia_node)>0)
    




print()
print("# total loop nodes: {}".format(len(list_loop_node)))
print("# total loops with collection<-collection: {}".format(sum(list_ia_node)))
print("# %loops with collection<-collection: {:.4f}".format(sum(list_ia_node)/len(list_loop_node)))

# processing 14624/14625
# total loop nodes: 14625
# total loops with collection<-collection: 2171
# %loops with collection<-collection: 0.1484
